## q-5

In this part implement regression with k-fold cross validation. Analyse how behav-
ior changes with different values of k. Also implement a variant of this which is the
leave-one-out cross validation.

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np
import sklearn as sk
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import linear_model

from pylab import *
import matplotlib
import matplotlib.pyplot as plt

###### loading dataset and preprocessing

In [2]:
df = pd.read_csv("../input_data/AdmissionDataset/data.csv")
X = df.drop(['Serial No.','Chance of Admit '],axis=1)
Y = df['Chance of Admit ']
col_names = [i for i in X]
X = pd.DataFrame(preprocessing.scale(X), columns = col_names)
X_copy = X.copy(deep=True)
Y_copy = Y.copy(deep=True)
X = X.values
Y = Y.values
n = df.shape[0]
df.shape

/home/priyendu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """


(450, 9)

In [3]:
theta = np.zeros([1,8])
alpha = 0.01
iterations = 1000

In [4]:
def gradientDescent(X,Y,theta,it,alpha,lamb):
    for i in range(it):
        theta = theta - (alpha/len(X)) * np.sum(X * (np.matmul(X, theta.T) - Y) + lamb*np.sign(theta), axis=0) 
    return theta

lamb = 0.01
# g = gradientDescent(X_train1,Y_train1,theta,iterations,alpha,lamb)
# theta_list = g[0]

In [5]:
def predict(X_test,theta_list):
    Y_pred=[]
    for index,row in X_test.iterrows():
        row=list(row)
        y1=0
        for i in range(1,8):
            y1=y1+theta_list[i]*row[i-1]
        y1=y1+theta_list[0]
        Y_pred.append(y1)
    return Y_pred
# pred = predict(X_test)

In [6]:
def getFolds(k,X):
    no_of_indices = len(X)/k
#     print no_of_indices
    start_indices = []
    t = 0
    while t<len(X):
        start_indices.append(t)
        t+=no_of_indices
    
    if len(start_indices) > k:
        start_indices = start_indices[:-1]
#     print start_indices
    
    test_fold = 0
    folds = []
    train_array = np.array([],dtype=int32)
    for i in xrange(k):
        train_array = []
        for s in xrange(len(start_indices)-1):
#             print start_indices[s],test_fold
            if s==test_fold:
#                 print "1 test ",start_indices[s], "to ",start_indices[s+1]
                test_array = np.arange(start_indices[s],start_indices[s+1],dtype=int32)
            else:
#                 print "1 train ",start_indices[s], "to ",start_indices[s+1]
                temp = np.arange(start_indices[s],start_indices[s+1],dtype=int32)
#                 print type(temp)
#                 temp_list = list(temp)
#                 print type(temp_list)
#                 print type(train_array)
#                 train_array.append(temp)
#                 train_array = train_array.flatten()
                train_array = np.append(train_array,temp)
                train_array = train_array.astype(np.int32)

                
#         print "adf ",len(start_indices)-1
        if test_fold == len(start_indices)-1:
#             print "2 test ",start_indices[-1], "to ",len(X)
            test_array = np.arange(start_indices[-1],len(X),dtype=int32)
        else:
#             print "2 train ",start_indices[-1], "to ",len(X)
            temp = np.arange(start_indices[-1],len(X),dtype=int32)
#             temp_list = list(temp)
#             train_array.append(temp)
            train_array = np.append(train_array,temp)
#             train_array = train_array.astype(np.int32)

#         train_array = np.array(train_array)
#         train_array = train_array.flatten()
        test_array = test_array.flatten()
        train_array = train_array.astype(np.int32)
        folds.append((train_array,test_array))
        test_fold+=1
    return folds

In [7]:
temp = np.ones(9)
temp.shape
getFolds(9,temp)


[(array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int32), array([0], dtype=int32)),
 (array([0, 2, 3, 4, 5, 6, 7, 8], dtype=int32), array([1], dtype=int32)),
 (array([0, 1, 3, 4, 5, 6, 7, 8], dtype=int32), array([2], dtype=int32)),
 (array([0, 1, 2, 4, 5, 6, 7, 8], dtype=int32), array([3], dtype=int32)),
 (array([0, 1, 2, 3, 5, 6, 7, 8], dtype=int32), array([4], dtype=int32)),
 (array([0, 1, 2, 3, 4, 6, 7, 8], dtype=int32), array([5], dtype=int32)),
 (array([0, 1, 2, 3, 4, 5, 7, 8], dtype=int32), array([6], dtype=int32)),
 (array([0, 1, 2, 3, 4, 5, 6, 8], dtype=int32), array([7], dtype=int32)),
 (array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32), array([8], dtype=int32))]

In [8]:
def KFolds(k):
#     kf = KFold(n_splits=k)
    kf = getFolds(k,X)
    mse_train = []
    mse_test = []
#     for train_index, test_index in kf.split(X):
    for train_index, test_index in kf:
#         print("TRAIN:", train_index, "TEST:", test_index)
#         print type(train_index[0])
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf = linear_model.Lasso(alpha=0.01)
        clf.fit(X_train, Y_train)

        z = np.ones((X_train.shape[0],1), dtype=float64)
        X_train = np.append(z,X_train,axis=1)
        Y_train = Y_train.reshape(Y_train.shape[0],1)

        g = gradientDescent(X_train,Y_train,theta,iterations,alpha,lamb)
        theta_list = g[0]
#         print theta_list
#         print 
#         print(clf.intercept_)  
#         print(clf.coef_)
#         print 
#         print

        X_test = pd.DataFrame(X_test)
        pred = predict(X_test,theta_list)
        mse_test.append(mean_squared_error(pred,Y_test))
        
        X_train = pd.DataFrame(X_train)
        pred = predict(X_train,theta_list)
        mse_train.append(mean_squared_error(pred,Y_train))
    return sum(mse_test) / len(mse_test), sum(mse_train) / len(mse_train) 

In [9]:
for i in xrange(2,16):
    mse_test, mse_train = KFolds(i)
    print "for k=",i," mse on test = ",mse_test," mse on train = ",mse_train


for k= 2  mse on test =  0.004094975394923045  mse on train =  0.008047101426706301
for k= 3  mse on test =  0.0040279890200014806  mse on train =  0.00787382626668405
for k= 4  mse on test =  0.004033307772314665  mse on train =  0.007862484797446492
for k= 5  mse on test =  0.004015653947012743  mse on train =  0.007824311432657107
for k= 6  mse on test =  0.004006436783423248  mse on train =  0.00783263604119621
for k= 7  mse on test =  0.004001047921338746  mse on train =  0.00783239432665564
for k= 8  mse on test =  0.0040162652932109865  mse on train =  0.007835418845015634
for k= 9  mse on test =  0.004013305003989613  mse on train =  0.007830007504049575
for k= 10  mse on test =  0.004033620177729699  mse on train =  0.007829647132570346
for k= 11  mse on test =  0.004009134425336771  mse on train =  0.007833101294798096
for k= 12  mse on test =  0.0039966047438229155  mse on train =  0.00782945244877821
for k= 13  mse on test =  0.00398476063532067  mse on train =  0.007829234

In [10]:
print "leave one out : "
mse_test, mse_train = KFolds(n)
print "for k=",n," mse on test = ",mse_test," mse on train = ",mse_train


leave one out : 
for k= 450  mse on test =  0.004046384420772772  mse on train =  0.007828103444295567


###### splitting data in training and validation

In [11]:
# X_train_c, X_test_c, Y_train_c, Y_test_c = train_test_split(X_copy, Y_copy, test_size=0.2)

###### using inbuilt linear regression model

###### appending a column of ones at the beginning

###### initializing parameters for gradient descent

###### method to calculate values of theta using gradient descent

###### method to predict values for test_data

In [12]:
# print theta_list
# print r2_score(list(Y_test),pred)

###### testing from file